In [1]:
from sklearn.datasets import load_digits
digits = load_digits()
X = digits.data
y = digits.target
digits.data.shape

(1797, 64)

In [2]:
import pandas as pd
import numpy as np

df = pd.read_csv('train.csv')
df.head()

,id,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,0,66,Private,216522,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,99999,0,55,?,>50K
1,1,53,?,256257,12th,8,Married-civ-spouse,?,Wife,White,Female,0,0,40,Italy,<=50K
2,2,44,Private,306075,10th,6,Divorced,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
3,3,73,Private,248575,Some-college,10,Married-civ-spouse,Prof-specialty,Husband,White,Male,0,2174,50,United-States,>50K
4,4,35,Self-emp-not-inc,420855,Some-college,10,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,60,United-States,<=50K


In [3]:
keys = ['age',
 'workclass',
 #'fnlwgt',
 'education',
 'education-num',
 'marital-status',
 'occupation',
 'relationship',
 'race',
 'sex',
 'capital-gain',
 'capital-loss',
 'hours-per-week',
 'native-country']
values = ['income <= 50K', 'income > 50K']

In [4]:
set_key = {}
def type2num(name,key):
    if name not in set_key:
        l_key = list(set(df[name]))
        set_key.setdefault(name,l_key)
    l_key = set_key[name]
    n = np.zeros(len(l_key))
    n[l_key.index(key)] = 1
    return list(n)
type2num('occupation','?')

[1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0]

In [5]:
df.describe()

,id,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week
count,30000.000000,30000.000000,3.000000e+04,30000.000000,30000.000000,30000.000000,30000.000000
mean,14999.500000,38.592933,2.906533e+05,10.084033,1088.225367,86.398867,40.437067
std,8660.398374,13.642750,1.055748e+05,2.573556,7423.000850,401.581917,12.329854
min,0.000000,17.000000,1.133090e+05,1.000000,0.000000,0.000000,1.000000
25%,7499.750000,28.000000,2.186980e+05,9.000000,0.000000,0.000000,40.000000
50%,14999.500000,37.000000,2.794005e+05,10.000000,0.000000,0.000000,40.000000
75%,22499.250000,48.000000,3.379022e+05,12.000000,0.000000,0.000000,45.000000
max,29999.000000,90.000000,1.585729e+06,16.000000,99999.000000,4356.000000,99.000000


In [6]:
# raw X
X = [[int(df['age'][i]), int(df['fnlwgt'][i])\
         , int(df['education-num'][i])\
         , int(df['capital-gain'][i]), int(df['capital-loss'][i]), int(df['hours-per-week'][i])]
         for i in range(len(df))]

In [7]:
# X with type
X = [[int(df['age'][i]), df['workclass'][i]
         #, int(df['fnlwgt'][i])\
         , df['education'][i]\
         , int(df['education-num'][i]), df['marital-status'][i], df['occupation'][i]\
         , df['relationship'][i], df['race'][i], df['sex'][i]\
         , int(df['capital-gain'][i]), int(df['capital-loss'][i]), int(df['hours-per-week'][i])\
         ,df['native-country'][i]]
         for i in range(len(df))]

In [8]:
all_num = 0
none_num = 0
for i in range(len(X)):
    for j in range(len(X[0])):
        all_num += 1
        if X[i][j] == '?':
            none_num += 1
print(all_num / len(X[0]),none_num)

30000.0 3946


In [9]:
X[0]

[66,
 'Private',
 'Bachelors',
 13,
 'Married-civ-spouse',
 'Exec-managerial',
 'Husband',
 'White',
 'Male',
 99999,
 0,
 55,
 '?']

In [10]:
y = []
for x in df['income']:
    if x == '<=50K':
        y.append(0)
    else:
        y.append(1)

In [11]:
def balance_data(X,y):
    b_X = {}
    for i in range(len(X)):
        b_X.setdefault(y[i],[])
        b_X[y[i]].append(X[i])
    lengths = [len(b_X[key]) for key in b_X]
    print(lengths)
    length_min = min(lengths)
    result_X, result_y = [],[]
    for key in b_X:
        result_X += b_X[key][:length_min]
        result_y += [key for i in range(length_min)]
    return result_X, result_y
X,y = balance_data(X,y)

[7239, 22761]


In [12]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train, y_test = train_test_split(X,y,test_size=0.40, random_state=0)
len(X_train),len(y_train)

(8686, 8686)

In [13]:
X,y = X_train,y_train

In [14]:


used_features = []
time_counter = {'init':0, 'find':0, 'seperate':0, 'pre_cut':0\
                , 'train':0, 'G_D_a':0, 'H_D':0, 'G_D_a_t_num':0, 'G_D_a_t_type':0}

def H_D(y):
    time_begin = time.time()
    r = 0
    l_y = len(y)
    log_2 = np.log(2)
    count = np.bincount(y)
    for i in set(y):
        r -= count[i]/l_y * np.log(count[i]/l_y) / log_2
    time_end = time.time()
    time_counter['H_D'] += time_end - time_begin
    return r

def T_a(y_a):
    l = [item for item in y_a]
    l.sort()
    l = [(l[i] + l[i + 1]) / 2 for i in range(len(l) - 1)]
    return l
    
def G_D_a_t_num(y,a,t,h_D):
    time_begin = time.time()
    pos_list, neg_list = [], []
    for i in range(len(a)):
        if a[i] > t:
            pos_list.append(y[i])
        else:
            neg_list.append(y[i])
    t = len(pos_list) / len(y) * H_D(pos_list)
    t += len(neg_list) / len(y) * H_D(neg_list)
    time_end = time.time()
    time_counter['G_D_a_t_num'] += time_end - time_begin
    return h_D - t

def G_D_a_t_type(y,a,h_D):
    time_begin = time.time()
    data_sep = {}
    t = 0
    for i in range(len(a)):
        data_sep.setdefault(a[i],[])
        data_sep[a[i]].append(y[i])
    for type_name in data_sep:
        t += len(data_sep[type_name]) / len(y) * H_D(data_sep[type_name])
    time_end = time.time()
    time_counter['G_D_a_t_type'] += time_end - time_begin
    return h_D - t

def a2type_by_t(a,t):
    a2type = []
    if type(a[0]) in [type(1),type(0.1)]:
        for item in a:
            if item < t:
                a2type.append(0)
            else:
                a2type.append(1)
    else:
        dic = list(set(a))
        for item in a:
            a2type.append(dic.index(item))
    return a2type
    
def G_ID3(y,a):
    time_begin = time.time()
    max_value = 0
    max_t = 0
    h_D = H_D(y)
    if type(a[0]) in [type(1),type(0.1)]:
        t_a = T_a(a)
        for t in t_a:
            v = G_D_a_t_num(y,a,t,h_D)
            if v > max_value:
                max_value = v
                max_t = t
    elif type(a[0]) == type('a'):
        max_value = G_D_a_t_type(y,a,h_D)
    time_end = time.time()
    time_counter['G_D_a'] += time_end - time_begin
    return max_value, max_t


In [15]:
def G_C45(y,a):
    max_value, max_t = G_ID3(y,a)
    a2type = a2type_by_t(a,max_t)
    h_a = H_D(np.array(a2type))
    return max_value / h_a, max_t
    

In [16]:
def G_CART_Y(y):
    count = np.bincount(y)
    l_y = len(y)
    t = 0
    for i in set(y):
        t += (count[i] * count[i]) /(l_y * l_y)
    return 1 - t
    
    
    
    
def G_CART(y,a):
    time_begin = time.time()
    min_value = 0
    min_t = 0
    h_D = H_D(y)
    if type(a[0]) in [type(1),type(0.1)]:
        t_a = T_a(a)
        for t in t_a:
            t_y_pos = []
            t_y_neg = []
            for i in range(len(a)):
                if a[i] < t:
                    t_y_pos.append(y[i])
                else:
                    t_y_neg.append(y[i])
            v = (len(t_y_pos) * G_CART_Y(t_y_pos) + len(t_y_neg) * G_CART_Y(t_y_neg)) / len(a)
            if v < min_value:
                min_value = v
                min_t = t
                
    elif type(a[0]) == type('a'):
        dic = {}
        for i in range(len(a)):
            dic.setdefault(a[i],[])
            dic[a[i]].append(y[i])
        v = 0
        for key in dic:
            v += len(dic[key]) * G_CART_Y(dic[key])
        min_value = v / len(a)
    time_end = time.time()
    time_counter['G_D_a'] += time_end - time_begin
    return 1 - min_value, min_t

In [17]:
import time

class Decision_tree():
    
    def __init__(self,G_method):
        self.G_method = G_method
        pass
    
    def train(self, X, y):
        time_begin = time.time()
        rest_indexs = list(range(len(X[0])))
        self.dtn = Decision_tree_node(rest_indexs, X, y, self.G_method,'FIRST')
        self.dtn.train()
        time_end = time.time()
        print('Training takes',time_end - time_begin,'s')
        
    def predict(self, X):
        y = [self.dtn.predict(x)[0] for x in X]
        return np.array(y)
    
    def draw(self,keys,values):
        self.dtn.draw(0,keys,values)
        
        
class Decision_tree_node():
    
    def __init__(self, rest, X, y, G_method,tag):
        time_begin = time.time()
        self.G_method = G_method
        self.using_index, self.t = self.find_best_index(rest, X, y)
        self.rest = rest
        self.X, self.y = X, y
        self.pos = None
        #print('\tadd\t',used_features,self.using_index)
        #print(used_features,self.using_index)
        used_features.append(self.using_index)
        time_end = time.time()
        time_counter['init'] += time_end - time_begin
        self.sub_trees = {}
        self.sub_values = {}
        self.tag = tag
    
    def find_best_index(self, rest, X, y):
        
        time_begin = time.time()
        max_index = -1
        max_value = -1
        max_t = 0
        #print(rest,used_features)
        for index in rest:
            if index in used_features:
                continue
            a, y_cleaned, data_weight_X = self.remove_not_known(X,y,index)
            if len(a) == 0:
                continue
            g, t = self.G_method(y_cleaned,a) 
            #print('g',g)
            if g > max_value:
                max_value = g
                max_index = index
                max_t = t
                self.sub_values = data_weight_X 
        
        time_end = time.time()
        time_counter['find'] += time_end - time_begin
        return max_index, max_t
    
    def remove_not_known(self, X, y, index):
        data_weight_X = {}
        data_value_X = []
        data_value_y = []
        for i in range(len(X)):
            if X[i][index] != '?':
                data_weight_X.setdefault(X[i][index],0)
                data_weight_X[X[i][index]] += 1
                data_value_X.append(X[i][index])
                data_value_y.append(y[i])
        for name in data_weight_X:
            data_weight_X[name] /= len(data_value_X)
        return data_value_X, np.array(data_value_y,dtype='int64'), data_weight_X
                
    def seperate_data_num(self, X, y):
        time_begin = time.time()
        X_pos, X_neg, y_pos, y_neg = [], [], [], []
        index, t = self.using_index, self.t
        for i in range(len(X)):
            if X[i][index] > t:
                X_pos.append(X[i])
                y_pos.append(y[i])
            else:
                X_neg.append(X[i])
                y_neg.append(y[i])
        time_end = time.time()
        time_counter['seperate'] += time_end - time_begin
        return X_pos, X_neg, y_pos, y_neg
    
    def seperate_data_type(self, X, y):
        time_begin = time.time()
        data_sepe_x = {}
        data_sepe_y = {}
        for i in range(len(X)):
            data_sepe_x.setdefault(X[i][self.using_index],[])
            data_sepe_y.setdefault(X[i][self.using_index],[])
            data_sepe_x[X[i][self.using_index]].append(X[i])
            data_sepe_y[X[i][self.using_index]].append(y[i])
        time_end = time.time()
        time_counter['seperate'] += time_end - time_begin
        return data_sepe_x, data_sepe_y
    
    def pre_cut(self):
        if len(self.X) < 5:
            count = np.bincount(np.array(self.y))
            self.value_major = np.argmax(count)
            print('pos',self.pos)
            return False
        type_x = []
        if type(self.X) in [type(0.1), type(1)]:
            for i in range(len(self.X)):
                if self.X[i] > self.t:
                    type_x.append('pos')
                else:
                    type_x.append('neg')
        else:
            type_x = self.X
        X_train,X_test, y_train, y_test = train_test_split(type_x,self.y,test_size=0.4, random_state=0)
        
        count = np.bincount(np.array(y_train))
        value_major = np.argmax(count)
        count_test_right = 0
        for i in range(len(y_test)):
            if y_test[i] == value_major:
                count_test_right += 1
        gener_per_not_cut = count_test_right / len(y_test)
        
        count_key_y = {}
        for i in range(len(X_train)):
            count_key_y.setdefault(X_train[i][self.using_index],[])
            count_key_y[X_train[i][self.using_index]].append(y_train[i])
        for key in count_key_y:
            c = np.array(count_key_y[key])
            count = np.bincount(c)
            value_c = np.argmax(count)
            count_key_y[key] = value_c
        
        count_test_right = 0
        for i in range(len(y_test)):
            #print(y_test,count_key_y)
            if X_test[i][self.using_index] in count_key_y and y_test[i] == count_key_y[X_test[i][self.using_index]]:
                count_test_right += 1
        gener_per_cut = count_test_right / len(y_test)
        
        self.value_major = value_major
        print(gener_per_cut,gener_per_not_cut)
        return gener_per_cut > gener_per_not_cut
        # return true if need cut
    
    def train(self):
        time_begin = time.time()
        if H_D(self.y) == 0 or self.using_index == -1:
            self.pos = self.y[0]
        else:
            if self.pre_cut():
                #print(1)
                if type(X[0][self.using_index]) in [type(1),type(0.1)]:
                    X_pos, X_neg, y_pos, y_neg = self.seperate_data_num(self.X, self.y)
                    if len(X_pos) > 0:
                        self.sub_trees['pos'] = Decision_tree_node(self.rest, X_pos, y_pos, self.G_method,'pos')
                        self.sub_trees['pos'].train()
                    #print(2)
                    if len(X_neg) > 0:
                        self.sub_trees['neg'] = Decision_tree_node(self.rest, X_neg, y_neg, self.G_method,'neg')
                        self.sub_trees['neg'].train()
                elif type(X[0][self.using_index]) == type('a'):
                    data_sepe_x, data_sepe_y = self.seperate_data_type(self.X, self.y)
                    #print('sep:',len(data_sepe_x),self.using_index)
                    for name in data_sepe_x:
                        if len(data_sepe_x[name]) > 0:
                            self.sub_trees[name] = Decision_tree_node(self.rest, data_sepe_x[name], data_sepe_y[name], self.G_method,data_sepe_x[name][0][self.using_index])
                            self.sub_trees[name].train()
            else:
                self.pos = self.value_major
                print('not cut')
        #print('\tremove\t',used_features,self.using_index)
        time_end = time.time()
        used_features.remove(self.using_index)
        time_counter['train'] += time_end - time_begin
    
    def predict(self,x):
        #print(self.using_index)
        if self.pos != None:
            return self.pos, 1
        elif type(x[self.using_index]) in [type(1),type(0.1)]:
            if x[self.using_index] > self.t and 'pos' in self.sub_trees:
                return self.sub_trees['pos'].predict(x)[0], self.sub_trees['pos'].predict(x)[1]
            elif x[self.using_index] <= self.t and 'neg' in self.sub_trees:
                return self.sub_trees['neg'].predict(x)[0], self.sub_trees['neg'].predict(x)[1]
        elif type(x[self.using_index]) == type('a'):
            if x[self.using_index] in self.sub_trees:
                return self.sub_trees[x[self.using_index]].predict(x)[0], self.sub_trees[x[self.using_index]].predict(x)[1]
        if x[self.using_index] == '?' or x[self.using_index] not in self.sub_trees :
            value_max = 0
            type_max = -1
            for name in self.sub_trees:
                tree = self.sub_trees[name]
                type_temp,value_temp = tree.predict(x)
                if name in self.sub_values:
                    value_temp *= self.sub_values[name]
                else:
                    value_temp = 0
                if value_temp  > value_max:
                    value_max = value_temp
                    type_max = type_temp
            return type_max, value_max
            
    def draw(self,level,keys,values):
        for i in range(level):
            print('|   ',end='')
        print('|---tag:',self.tag,end='')
        if self.pos == None:
            print('')
            for i in range(level):
                print('|   ',end='')
            print('|   division:',keys[self.using_index],end='')
            if self.t != 0:
                print(' >?',self.t)
            else:
                print('')
            for name in self.sub_trees:
                self.sub_trees[name].draw(level + 1,keys,values)
        else:
            print(' -----class:',values[self.pos])
            

        
        
        

In [18]:
from sklearn.model_selection import KFold

class Mult_Decision_tree():
    
    def __init__(self, num = 5):
        self.num = num
    
    def train(self, X, y):
        kf = KFold(n_splits = self.num)
        self.dts = []
        for train, test in kf.split(X):
            X_kf = [X[i] for i in train]
            y_kf = [y[i] for i in train]
            dt = Decision_tree()
            dt.train(X_kf,y_kf)
            self.dts.append(dt)
        
    def predict(self,x):
        results = []
        result = []
        for dt in self.dts:
            results.append(dt.predict(x))
        for i in range(len(x)):
            value = {}
            for r in results:
                value.setdefault(r[i],0)
                value[r[i]] += 1
            value_max = max(value, key=value.get)
            result.append(value_max)
        return np.array(result)
            



In [19]:
from sklearn.metrics import classification_report
dt_id3 = Decision_tree(G_ID3)
dt_id3.train(X,y)

0.7602877697841727 0.5064748201438849
0.7485062466051059 0.7164584464964693
0.6147540983606558 0.6024590163934426
0.5711111111111111 0.5755555555555556
not cut
0.9154929577464789 0.9436619718309859
not cut
0.8861047835990888 0.8906605922551253
not cut
0.7017543859649122 0.6871345029239766
0.6470588235294118 0.6241830065359477
0.4117647058823529 0.5882352941176471
not cut
0.42424242424242425 0.5606060606060606
not cut
0.25 0.375
not cut
0.52 0.72
not cut
0.8305084745762712 0.8305084745762712
not cut
0.625 0.5
0.0 0.42857142857142855
not cut
0.4375 0.5
not cut
0.5454545454545454 0.7727272727272727
not cut
0.25 0.5625
not cut
0.21428571428571427 0.7142857142857143
not cut
0.5 0.7222222222222222
not cut
0.875 0.875
not cut
1.0 0.6666666666666666
0.0 1.0
not cut
0.8 0.8
not cut
0.9230769230769231 0.9423076923076923
not cut
0.5833333333333334 0.4166666666666667
0.0 0.75
not cut
pos None
not cut
pos None
not cut
pos None
not cut
0.6774193548387096 0.6451612903225806
0.2857142857142857 0.14285

In [20]:
y_pred,y_true = dt_id3.predict(X_test), y_test
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.84      0.73      0.78      2928
           1       0.76      0.86      0.80      2864

    accuracy                           0.79      5792
   macro avg       0.53      0.53      0.53      5792
weighted avg       0.80      0.79      0.79      5792



/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/metrics/classification.py:1439: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [21]:
y_pred

array([0, 1, 1, ..., 1, 0, 1])

In [22]:
dt_id3.draw(keys,values)

|---tag: FIRST
|   division: relationship
|   |---tag: Husband
|   |   division: education
|   |   |---tag: HS-grad
|   |   |   division: capital-gain >? 5013.0
|   |   |   |---tag: pos -----class: income > 50K
|   |   |   |---tag: neg -----class: income > 50K
|   |   |---tag: Prof-school -----class: income > 50K
|   |   |---tag: Bachelors -----class: income > 50K
|   |   |---tag: Some-college
|   |   |   division: capital-gain >? 5013.0
|   |   |   |---tag: pos -----class: income > 50K
|   |   |   |---tag: neg
|   |   |   |   division: occupation
|   |   |   |   |---tag: Adm-clerical -----class: income > 50K
|   |   |   |   |---tag: Craft-repair -----class: income > 50K
|   |   |   |   |---tag: ? -----class: income > 50K
|   |   |   |   |---tag: Sales -----class: income > 50K
|   |   |   |   |---tag: Exec-managerial -----class: income > 50K
|   |   |   |   |---tag: Handlers-cleaners
|   |   |   |   |   division: workclass
|   |   |   |   |   |---tag: Federal-gov -----class: income > 5

In [23]:
dt_c45 = Decision_tree(G_C45)
dt_c45.train(X,y)

0.5942446043165468 0.5064748201438849


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: invalid value encountered in double_scalars
  """


0.9736070381231672 0.9736070381231672
not cut
0.6458200382897256 0.5513720485003191
0.7274502147598594 0.5365091761030847
0.7447058823529412 0.71
0.8378728923476005 0.8495460440985733
not cut
0.578494623655914 0.5709677419354838
0.803921568627451 0.8823529411764706
not cut
0.5642775881683731 0.5620022753128555
0.5971867007672634 0.5907928388746803
0.6406685236768802 0.6086350974930362
0.6105263157894737 0.6
0.6011396011396012 0.6096866096866097
not cut
0.0 0.6666666666666666
not cut
0.6190476190476191 0.42857142857142855
0.5 0.7142857142857143
not cut
0.0 0.6666666666666666
not cut
pos None
not cut
pos None
not cut
0.6705426356589147 0.6317829457364341
0.6956521739130435 0.8260869565217391
not cut
0.6785714285714286 0.6964285714285714
not cut
0.6 0.6
not cut
0.8301886792452831 0.8301886792452831
not cut
0.6 0.6
not cut
0.575 0.65
not cut
0.125 0.875
not cut
0.23076923076923078 0.7692307692307693
not cut
0.6666666666666666 0.75
not cut
0.6666666666666666 0.6666666666666666
not cut
0.857

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


0.921602787456446 0.9233449477351916
not cut
Training takes 404.327800989151 s


In [24]:
y_pred,y_true = dt_c45.predict(X_test), y_test
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.81      0.79      0.80      2928
           1       0.79      0.81      0.80      2864

    accuracy                           0.80      5792
   macro avg       0.54      0.53      0.53      5792
weighted avg       0.80      0.80      0.80      5792



In [31]:
dt_c45.draw(keys,values)

|---tag: FIRST
|   division: capital-gain >? 5013.0
|   |---tag: pos -----class: income > 50K
|   |---tag: neg
|   |   division: age >? 27.0
|   |   |---tag: pos
|   |   |   division: marital-status
|   |   |   |---tag: Married-civ-spouse
|   |   |   |   division: education-num >? 10.0
|   |   |   |   |---tag: pos -----class: income > 50K
|   |   |   |   |---tag: neg
|   |   |   |   |   division: capital-loss >? 1740.0
|   |   |   |   |   |---tag: pos -----class: income > 50K
|   |   |   |   |   |---tag: neg
|   |   |   |   |   |   division: hours-per-week >? 36.0
|   |   |   |   |   |   |---tag: pos
|   |   |   |   |   |   |   division: native-country
|   |   |   |   |   |   |   |---tag: United-States
|   |   |   |   |   |   |   |   division: education
|   |   |   |   |   |   |   |   |---tag: HS-grad
|   |   |   |   |   |   |   |   |   division: race
|   |   |   |   |   |   |   |   |   |---tag: White -----class: income > 50K
|   |   |   |   |   |   |   |   |   |---tag: Amer-Indian-Esk

In [25]:
from sklearn.metrics import classification_report
dt_cart = Decision_tree(G_CART)
dt_cart.train(X,y)

0.6633093525179856 0.5064748201438849
0.6667625899280576 0.5064748201438849
0.5942446043165468 0.5064748201438849
0.8473451327433629 0.8473451327433629
not cut
0.601389348329474 0.5527621567978829
0.7083333333333334 0.7592592592592593
not cut
0.6463675213675214 0.5787037037037037
0.7549857549857549 0.5787037037037037
0.701867816091954 0.6479885057471264
0.5461346633416458 0.5311720698254364
0.5 0.5277777777777778
not cut
0.631578947368421 0.631578947368421
not cut
0.5263157894736842 0.47368421052631576
0.5 0.5
not cut
0.8888888888888888 0.8888888888888888
not cut
0.4 0.6
not cut
0.5 0.5192307692307693
not cut
0.5116279069767442 0.5581395348837209
not cut
0.3333333333333333 0.7333333333333333
not cut
0.625 0.75
not cut
0.7674418604651163 0.7209302325581395
0.6818181818181818 0.6818181818181818
not cut
0.7 0.7
not cut
0.2857142857142857 0.2857142857142857
not cut
0.5 0.75
not cut
0.47619047619047616 0.47619047619047616
not cut
0.5714285714285714 0.6428571428571429
not cut
0.625 0.625
not

In [26]:
y_pred,y_true = dt_cart.predict(X_test), y_test
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.86      0.69      0.76      2928
           1       0.74      0.88      0.80      2864

    accuracy                           0.78      5792
   macro avg       0.53      0.52      0.52      5792
weighted avg       0.80      0.78      0.78      5792



In [27]:
class Merge_tree:
    def __init__(self,models):
        self.models = models
    def predict(self,x):
        results = []
        result = []
        for dt in self.models:
            results.append(dt.predict(x))
        for i in range(len(x)):
            value = {}
            for r in results:
                value.setdefault(r[i],0)
                value[r[i]] += 1
            value_max = max(value, key=value.get)
            result.append(value_max)
        return np.array(result)

In [28]:
mt = Merge_tree([dt_id3, dt_c45, dt_cart])
y_pred,y_true = mt.predict(X_test), y_test
print(classification_report(y_true, y_pred))

              precision    recall  f1-score   support

          -1       0.00      0.00      0.00         0
           0       0.86      0.73      0.79      2928
           1       0.76      0.87      0.81      2864

    accuracy                           0.80      5792
   macro avg       0.54      0.54      0.53      5792
weighted avg       0.81      0.80      0.80      5792



In [29]:
mct = Mult_Decision_tree(20)
mct.train(X,y)
y_pred,y_true = mct.predict(X_test), y_test
print(classification_report(y_true, y_pred))

TypeError: __init__() missing 1 required positional argument: 'G_method'

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
dtc = DecisionTreeClassifier()
dtc.fit(X,y)
y_pred,y_true = dtc.predict(X_test,), y_test
print(classification_report(y_true, y_pred))

In [ ]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X,y)
y_pred,y_true = xgb.predict(X_test,), y_test
print(classification_report(y_true, y_pred))